# Duck iris prediction

In [2]:
%load_ext magic_duckdb

In [3]:
import duckdb

# Open a connection to your DuckDB database
con = duckdb.connect("my-data.duckdb")

In [4]:
%%dql -co con
CREATE TABLE IF NOT EXISTS birds AS SELECT * FROM read_csv('../../data/birds.csv');
CREATE TABLE IF NOT EXISTS ducks AS SELECT * FROM read_csv('../../data/ducks.csv');

,Count


In [5]:
%%dql -co con
-- create raw training data
CREATE OR REPLACE TABLE prepared_data AS
SELECT
    Species_Common_Name as species,
    Beak_Width AS beak_width,
    Beak_Length_Culmen AS beak_length,
    Wing_Length AS wing_length,
    Tail_Length AS tail_length
FROM birds
INNER JOIN ducks ON name = Species_Common_Name
WHERE beak_width is not null and beak_length is not null and wing_length is not null and tail_length is not null
and species like 'Harlequin%' or species like 'Egyptian%' or species like 'Pink%' or species like 'King%' or species like 'Muscovy%'
or species like 'Fueg%' or species like 'Smew%' or species like 'Laysan%';

-- create target mapping
CREATE OR REPLACE TABLE target_mapping AS
SELECT species,
       ROW_NUMBER() OVER (ORDER BY species) - 1 AS target
FROM (SELECT DISTINCT species FROM prepared_data);

-- create ml features
CREATE OR REPLACE TABLE ml_data AS
SELECT
    t.target as y,
    [p.beak_width, p.beak_length, p.wing_length, p.tail_length] as X
FROM prepared_data p
JOIN target_mapping t ON p.species = t.species;

,Count
0,50


# SQL Sampling

In [8]:
%%dql -co con
ALTER TABLE ml_data ADD COLUMN IF NOT EXISTS random_value DOUBLE;
UPDATE ml_data SET random_value = random();

,Count
0,50


In [9]:
%%dql -co con
CREATE OR REPLACE TABLE train_data AS
WITH stratified_data AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY y ORDER BY random_value) AS row_num,
           COUNT(*) OVER (PARTITION BY y) AS class_count
    FROM ml_data
)
SELECT
    y, X
FROM stratified_data
WHERE row_num <= CEIL(0.65 * class_count);

,Count
0,34


In [10]:
%%dql -co con
CREATE OR REPLACE TABLE test_data AS
SELECT
    y, X
FROM ml_data
EXCEPT
SELECT
    y, X
FROM train_data;

,Count
0,16


In [15]:
df = con.execute("""SELECT * FROM train_data ORDER BY y""").fetchnumpy()
X = df['X'].tolist()
y = df['y'].tolist()

In [16]:
# sklearn training
from sklearn import tree
classifier=tree.DecisionTreeClassifier()
classifier.fit(X,y)

# Register sklearn model in Python UDF
def predict_duck(X):
    pred = classifier.predict([X])[0]
    return int(pred)

con.create_function("predict_duck", predict_duck, ['DOUBLE[]'], 'INTEGER')

/var/folders/r7/h8c47kyx6z3d9f_0gwz30jmw0000gq/T/ipykernel_9039/1005867466.py:12: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  con.create_function("predict_duck", predict_duck, ['DOUBLE[]'], 'INTEGER')


In [17]:
%%dql -co con
FROM test_data ORDER BY y

,y,X
0,0,"[22.2, 53.0, 403.0, 140.0]"
1,0,"[20.5, 48.0, 350.0, 112.0]"
2,1,"[31.4, 61.4, 264.0, 97.0]"
3,2,"[11.0, 25.9, 187.0, 80.0]"
4,2,"[13.2, 29.7, 201.0, 88.0]"
5,3,"[17.2, --, 253.0, 76.0]"
6,3,"[19.0, 34.6, 269.0, 77.0]"
7,4,"[14.4, 44.0, 198.0, 70.0]"
8,4,"[14.6, 46.0, 213.0, 72.0]"
9,5,"[23.9, 71.1, 323.0, 190.0]"


In [18]:
%%dql -co con
-- make predictions on test data
CREATE OR REPLACE TABLE predictions AS SELECT y, predict_duck(X) as y_pred FROM test_data

,Count
0,16


In [19]:
%%dql -co con
-- evaluate model on test sample
SELECT
    COUNT(*) AS total_predictions,
    SUM(CASE WHEN y = y_pred THEN 1 ELSE 0 END) AS correct_predictions,
    CAST(SUM(CASE WHEN y = y_pred THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) AS accuracy
FROM predictions;

,total_predictions,correct_predictions,accuracy
0,16,14.0,0.875
